# **Tutorial for DICStateField model component**

DICStateField takes the displacement history as an input to the calculation of the 
strain, damage and state fields on an interpolated grid in the **load-space-time** databasis.

In [ ]:
%matplotlib widget
import matplotlib.pylab as plt
import numpy as np
from bmcs_shear.api import DICGrid, DICInpUnstructuredPoints, DICStateFields
import sz_tests_series_2023 as ts

The input to the simulation is the `DICGrid` instance providing the history of the displacement field for a grid of markers. 

An instance of the state field model component is constructed as

In [ ]:
dcl = ts.new_dcl(ts.B10_TV2)
dsf = dcl.dsf

In [ ]:
fig, ax = plt.subplots(figsize=(9,3))
dsf.dic_grid.t = 1.4
dsf.plot_crack_detection_field(ax, fig)

In [ ]:
dsf.R_MN, dsf.n_M, dsf.n_N, dsf.R, dsf.n_irn_M, dsf.n_irn_N

In [ ]:
fig, ax = plt.subplots(figsize=(9,3))
dsf.dic_grid.t = 1
cax_neg = fig.add_axes([0.1, 0.1, 0.4, 0.05])
cax_pos = fig.add_axes([0.5, 0.1, 0.4, 0.05])
dsf.plot_eps_MNab(ax, cax_neg, cax_pos)

The displacement-driven simulation of the the damage evolution using an anisotropic damage model is started using the `eval` method

The calculated fields can then be rendered using the `interact` methods

In [ ]:
import numpy as np
from scipy import signal

def autocorr(x):
    result = np.correlate(x, x, mode='full')
    return result[result.size // 2:]

def autocorr_length(x, x_mean):
    ac = autocorr(x - x_mean)
    # ac = autocorr(x - np.average(x))
    ac = ac / np.max(ac)    # Normalize
    return np.where(ac < 0)[0][0]

def fit_eps_poly(x, eps):
    # Fit a quadratic function to the data
    n_coeffs = 7
    coefficients = np.polyfit(x, eps, n_coeffs)

    # You can create a function for the fitted line like this:
    fitted_function = np.poly1d(coefficients)

    # You can then use this function to calculate the fitted values
    eps_fit = fitted_function(x)

    return eps_fit

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(12,4))
X_L = dsf.X_IJa[:,-2,0]
eps_L = dsf.eps_TIJab[-1,:,-2,0,0].T
ax.plot(X_L, eps_L, color='blue')
eps_glb_L = fit_eps_poly(X_L, eps_L)
ax.plot(X_L, eps_glb_L, color='red')
d_X = X_L[1] - X_L[0]
corr_length = autocorr_length(eps_L, eps_glb_L) * d_X
print(f'Estimated autocorrelation length: {corr_length:.1f} mm')

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(12,4))
y_idx = -1
X_L = dsf.X_IJa[:,y_idx,0]
U_L = dsf.dic_grid.U_TIJa[-3,:,y_idx,0].T
ax.plot(X_L, U_L, color='blue')
U_glb_L = fit_eps_poly(X_L, U_L)
ax.plot(X_L, U_glb_L, color='red')
d_X = X_L[1] - X_L[0]
corr_length = autocorr_length(U_L, U_glb_L) * d_X
print(f'Estimated autocorrelation length: {corr_length:.1f} mm')

In [ ]:
x0, y0, x1, y1 = dsf.dic_grid.X_frame
dsf.dic_grid.L_x / dsf.R / 5


In [ ]:
x_JK, y_JK = np.einsum('IJa->aIJ', dsf.dic_grid.X_IJa)
x_MN, y_MN = np.einsum('MNa->aMN', dsf.X_ipl_MNa)
U_JKa = dsf.dic_grid.U_IJa
U_MNa = dsf.get_z_MN_ironed(x_JK, y_JK, U_JKa, 70, x_MN, y_MN)

In [ ]:
U_MNa

In [ ]:
dsf.dic_grid.t

In [ ]:
np.max(max_eps_KL), np.average(max_eps_KL)

In [ ]:
np.max(max_sig_KL), np.average(max_sig_KL)

In [ ]:
dsf.interact()

In [ ]:
X0_a = np.array([40, 20])
X1_a = np.array([600, 20])
xi_p = np.linspace(0,1,500)
dX_a = X1_a - X0_a

In [ ]:
X_pa = X0_a[np.newaxis, :] + dX_a[np.newaxis,:] * xi_p[:, np.newaxis]
x_p, y_p = X_pa.T
t_p = 1 * np.ones_like(x_p)

In [ ]:
%matplotlib widget
import matplotlib.pylab as plt
fig, ax = plt.subplots(1,1)
fig.canvas.header_visible = False
u_0, u_1 = dsf.f_U_ipl_txy((t_p, x_p, y_p)).T
ax.plot(xi_p, u_0)

In [ ]:
dsf.f_eps_fe_txy((0.5, 200, 200))

In [ ]:
self = dsf
t_TMN, X_TMN, Y_TMN = self.mgrid_ipl_TMN
txy = np.c_[t_TMN.flatten(), X_TMN.flatten(), Y_TMN.flatten()]
eps_ab_txy = self.f_eps_fe_txy(txy)
eps_TMNab = eps_ab_txy.reshape(self.n_ipl_T, self.n_ipl_M, self.n_ipl_N, 2, 2)
eps_TMNa, eps_ev_TMNab = np.linalg.eig(eps_TMNab)
eps_ev_TMN0b = eps_ev_TMNab[...,0,:]

In [ ]:
self = dsf
t_TMN, X_TMN, Y_TMN = self.mgrid_ipl_TMN
txy = np.c_[t_TMN.flatten(), X_TMN.flatten(), Y_TMN.flatten()]
kappa_txy = self.f_kappa_fe_txy(txy)
kappa_TMN = kappa_txy.reshape(self.n_ipl_T, self.n_ipl_M, self.n_ipl_N)

In [ ]:
%matplotlib widget
fig, (ax, ax_eps, ax_kappa) = plt.subplots(3,1)
x_MN, y_MN = np.einsum('...a->a...', dsf.X_ipl_MNa)
T = -1
eps_scale_TM = eps_TMNa[T,:,:,0]
eps_ridge_TM = np.copy(eps_scale_TM)
eps_cr = 0.0001
eps_ridge_TM[np.where(eps_ridge_TM < eps_cr)] = 0
max_eps_ridge = np.max(eps_ridge_TM)
eps_ridge_TM /= max_eps_ridge
ax_eps.contourf( x_MN, y_MN, eps_ridge_TM, cmap='BuPu')
ax_eps.axis('equal')
ax_eps.axis('off');
contour_levels = np.linspace(0,1,10)
ax.contourf( x_MN, y_MN, dsf.omega_irn_TMN[T], contour_levels, cmap='BuPu')
#eps_scale[np.where(eps_scale > 0.003)] = 0
eps_scale_TM[np.where(eps_scale_TM < eps_cr)] = 0
eps_u_MN, eps_v_MN = np.einsum('...a->a...', eps_ev_TMN0b[T]) * eps_scale_TM
ax.quiver( x_MN, y_MN, eps_u_MN, eps_v_MN, angles='xy', pivot='middle')
ax.quiver( x_MN, y_MN, -eps_u_MN, -eps_v_MN, angles='xy', pivot='middle')
ax.axis('equal')
ax.axis('off');
ax_kappa.contourf( x_MN, y_MN, kappa_TMN[T], levels=[0.002, 0.0022, 0.02, 0.05], cmap='BuPu')
ax_kappa.axis('equal')
ax_kappa.axis('off');


## Grid of finite element quadrature points

The calculation of the strain, stress and damage fields using the finite elements returns the values at the quadrature points of the rectangles. The global positions of these points are available via the property attribute `X_fe_KLa`. As an example, the corner point coordinates can be accessed as

In [ ]:
dsf.X_fe_KLa[(0, 0, -1, -1),(0, -1, 0, -1), :]

Note that the grid `X_fe_KLa` has irregular spacing given by the optimal positioning of the quadrature points within the bilinear finite elements. This can be seen by slicing the first four markers along the first horizontal row $L=0$ and printing the $x$ coordinate $a=0$

In [ ]:
dsf.X_fe_KLa[:4, 0, 0]

A regular interpolation grid is provided within the frame covered by $X^\mathrm{fe}_{KLa}$ as $X^\mathrm{ipl}_{MNa}$. The same slice along the four markers of the bottom row renders positions with constant spacing as 

In [ ]:
dsf.X_ipl_MNa[:4, 0, 0]

## Strain field

The strain field at ultimate load arranged in correspondence to the $K, L$ grid is obtained using the `eps_fields` property, which delivers a tuple with the fields
 - `eps_Emab` - strain tensor $\varepsilon_{ab}$ with with $a,b \in (0,1,2)$ iin each quadrature point $m \in (0, 1, 2, 3)$ of an element $E \in (0, n_E)$ 
 - `eps_KLab` - strain tensor $\varepsilon_{ab}$ with with $a,b \in (0,1,2)$ iin each quadrature grid point with global horizontal index $K \in n_K$ and vertical index $L \in n_L$
 - `eps_KLa` - principal strain $\varepsilon_a$ with $a \in (0,1,2)$ in each quadrature grid point with global horizontal index $K \in n_K$ and vertical index $L \in n_L$
 - `max_eps_KL` - maximal strain in each quadrature grid point with global horizontal index $K \in n_K$ and vertical index $L \in n_L$

In [ ]:
eps_Emab, eps_KLab, eps_KLa, max_eps_KL = dsf.eps_fe_fields

The principal stresses in the corner points are obtained as

In [ ]:
eps_KLa[(0, 0, -1, -1),(0, -1, 0, -1), :]

The maximum and minimum principal strain values are obtained using numpy.max method 

In [ ]:
np.max(eps_KLa), np.min(eps_KLa)

The history of strain tensor is accessible via the five dimensional array `eps_fe_TKLab`. The last value of strain in the bottom left gauss point can be accessed via 

In [ ]:
dsf.eps_fe_TKLab[-1,0,0,:]

### Strain interpolator within the load-space domain

To obtain the strain tensor at an arbitrary point of the load-space domain an interpolater over the data points provided for the indexes $T, M, N$ in load, horizontal and vertical dimensions named `f_eps_ipl_txy([t, x, y])` with $t \in (0,1)$.

In [ ]:
dsf.f_eps_fe_txy(np.array([0.3, 100, 30]))

In [ ]:
# Damage field

The access to damage values in the grid of quadrature points is done in analogy to the strain. To get the variants of the damage tensor field $\omega_{ab}$ we can access the property attribute

In [ ]:
dsf.f_omega_fe_txy(np.array([[1, 100, 30], [1, 1260.0, 30]]))

In [ ]:
t_TMN, x_TMN, y_TMN = dsf.mgrid_ipl_TMN
t_factor = (x_TMN[0, -1, -1] + y_TMN[0, -1, -1]) / 2

# Throughout the history, damage can only grow

In some cases, damage indicator might diminish between the time steps.
The reason is not yet completely clear. Below is the technique how
to ensure an ascending history of damage for the case that it drops
between the imposed DIC steps.

In [ ]:
omega_TMN = dsf.omega_ipl_TMN

In [ ]:
o_a = np.array([[10, 9, 4, 1, 0],
                [10, 9, 4, 2, 0],
                [10, 8, 0, 3, 0]])
for T in range(1, len(o_a)):
    a_change = np.where(o_a[-T-1] > o_a[-T])
    o_a[-T-1,a_change] = o_a[-T,a_change]
o_a

In [ ]:
omega_asc_TMN = np.copy(omega_TMN)
for T in np.arange(1, dsf.n_ipl_T):
    # print(T)
    MN_change = np.where(omega_asc_TMN[-T-1,...] > omega_asc_TMN[-T,...])
    TMN0_change = (np.ones_like(MN_change[0])*(-T-1),) + MN_change
    TMN1_change = (np.ones_like(MN_change[0])*(-T),) + MN_change
    # print(MN_change[0].shape)
    omega_asc_TMN[TMN0_change] = omega_asc_TMN[TMN1_change]

# Smoothing algorithm

In [ ]:
# from mayavi import mlab
# mlab.contour3d(t_TMN * t_factor, x_TMN, y_TMN, omega_asc_TMN, contours=[0.75, 0.85, 0.92])
# mlab.show()

In [ ]:
dsf.R = 9

In [ ]:
RR = dsf.R
delta_x_MN = x_TMN[None, None, 0, ...] - x_TMN[0, ..., None, None]
delta_y_MN = y_TMN[None, None, 0, ...] - y_TMN[0, ..., None, None]
r2_MNOP = (delta_x_MN ** 2 + delta_y_MN ** 2) / (2 * RR ** 2)
alpha_r_MNOP = np.exp(-r2_MNOP)
a_OP = np.trapz(np.trapz(alpha_r_MNOP, x_TMN[0, :, 0], axis=-2), y_TMN[0, 0, :], axis=-1)
normed_a_MNOP = np.einsum('MNOP,MN->MNOP', alpha_r_MNOP, 1 / a_OP)
omega_TMNOP = np.einsum('MNOP,TOP...->TMNOP...', normed_a_MNOP, omega_asc_TMN)

In [ ]:
Omega_TMNP_y = np.trapz(omega_TMNOP, y_TMN[0, 0, :], axis=-1)

In [ ]:
Omega_TMN = np.trapz(Omega_TMNP_y, x_TMN[0, :, 0], axis=-1)

In [ ]:
for T in np.arange(dsf.n_ipl_T, 1, -1)-1:
    print(T, T-1)
    MN_change = np.where(Omega_TMN[T,...] < Omega_TMN[T-1,...])
    Omega_TMN[T-2, MN_change] = Omega_TMN[T, MN_change]

In [ ]:
np.max(Omega_TMN)

In [ ]:
# note that the inner integral cancels the dimension J on the axis with
# index 2. Therefore, the outer integral integrates over K - again on
# the axis with index 2
# omega_ipl_TMN = np.trapz(np.trapz(omega_TMNOP, x_TMN[:, :, 0], axis=3), y_TMN[:, 0, :], axis=3)


In [ ]:
x_NM = np.einsum('TMN->NMT', x_TMN)[...]
y_NM = np.einsum('TMN->NMT', y_TMN)[...]
z_NM = np.einsum('TMN->NMT', t_TMN)[...] * t_factor

In [ ]:
# from mayavi import mlab
# mlab.contour3d(t_TMN * t_factor, x_TMN, y_TMN, omega_asc_TMN, contours=[0.75, 0.85, 0.92])
# mlab.show()

# Interpolators

Interpolators are provided in form of properties
- The interpolation of the displacements is done on the `DIC` grid with indexes denoted $I,J$.  
- Interpolation of the strain, stress and damage fields is done on the `FE` quadrature grid $M, N$.
- Interpolaters on a finer, interpolated regular grid `IPL` with indexes $M, N$.